# Data Science Component Building 
### time series analysis, classification of headlines,topic modelling and sentimenta analysis and predictive analysis and modelling among various global media agencies

This section aims to build various models for our data inorder to gain a deeper understanding. Using the insights we gained from the EDA, we create various pipelines.


## Tools

I'll be using the library like nltk, sklearn, gensim and various other libraries to facilitate in creating various models


In [ ]:
# preparing and importing the necessary packages 
import pandas as pd
import numpy as np
import sys
import os
# Manually setting the correct path to src directory
sys.path.append("../../../OFFICAL WEEK0")

from src.loader import NewsDataLoader
import src.utils_ml as utils_ml
import src.utils as utils


In [ ]:
# defining the appropriate path for each path
path = {
    "rating":"../../data/cleaned_rating.csv",
    "domains_location":os.environ['DOMAINS_LOCATION_PATH'],
    "traffic":"../../data/cleaned_traffic.csv"}

# initiating the newsdataloader class
data_loader = NewsDataLoader(path)

In [ ]:
# importing relevant data by using the NewsDataloader class 
rating = data_loader.get_data("rating")

domains_location = data_loader.get_data("domains_location")

traffic = data_loader.get_data("traffic")